# Noms du binome: CILLEROS JEAN

# Prénoms du binome: Victor Jérémy

## Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe spécifique qui générera un mot de passe en prennant **8 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [1]:
import numpy as np

from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre)
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [2]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values



Q: Quelles sont les 5 lettres les plus représentées ?

In [13]:
# TO DO
monogramme.sort_values(by='frequency', ascending=False).letters.iloc[:5]

0    E
1    S
2    A
3    N
4    T
Name: letters, dtype: object

Les 5 lettres les plus représentées sont : 
E S A N T

Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien "gérer le cas ou la probabilité est nulle).

In [14]:
def entropie(freq):
    ent = 0
    for f in freq:
        if f>0:
            ent -= f*np.log2(f)
    return ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [18]:
# TO DO
print(f"L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : {round(8*entropie(monogramme['frequency'].tolist()),4)} bits.")

L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : 31.6762 bits.


Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [37]:


# TO DO
t_100000=0
letters = monogramme.letters.tolist()
proba=monogramme.frequency.tolist()
nbEssais=100_000
for __ in range(10):
    t = time.time()
    essai= np.random.choice(letters,p=proba,size=(nbEssais,8))
    t_100000 += time.time() - t
print(t_100000/10, 's')
tempsref = t_100000/10

0.045971465110778806 s


Ici calcul de ce temps est une moyenne sur 10 essais 

Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [35]:
# TO DO
H=8*entropie(monogramme['frequency'].tolist())
minG=(2**H/4)+1
print(int(minG))

857904864


Q: combien de temps cela prendra-t-il pour générer un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [47]:
# TO DO
print(f"Il faut en moyenne : {((tempsref*minG)/100000)/60} minutes ")


Il faut en moyenne : 6.573190592511765 minutes 


On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [39]:

bigramme = read_csv('bigramme.csv',keep_default_na=False)


print(bigramme.head(5))
freq_bi = (bigramme['frequency']).values

letters_bi = (bigramme['letters']).values



  letters  frequency
0      AA   0.000252
1      AB   0.001760
2      AC   0.003482
3      AD   0.001960
4      AE   0.000200


Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [40]:
# TO DO
bigramme.sort_values(by='frequency', ascending=False).letters.iloc[:5]

122    ES
117    EN
82     DE
290    LE
357    NT
Name: letters, dtype: object

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [41]:
# TO DO
# TO DO
print(f"L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : {round(4*entropie(bigramme['frequency'].tolist()),4)} bits.")

L'entropie pour un mot de passe de 8 lettres avec cde modèle probabiliste est minimale pour : 30.1423 bits.


Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?

D'après le cours (entropie conditionnelle) on a : $H(X,Y) <= H(X) + H(Y)$ 

Ainsi on a H(X,X) <= 2*H(X)

Nécessairement 4*H(X,X) <= 8*H(X)

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [42]:
# TO DO
t_100000=0
letters = bigramme.letters.tolist()
proba=bigramme.frequency.tolist()
nbEssais=100_000
for __ in range(10):
    t = time.time()
    essai= np.random.choice(letters,p=proba,size=(nbEssais,4))
    t_100000 += time.time() - t
print(t_100000/10, 's')
tempsrefBigramme = t_100000/10


0.042973566055297854 s


Q: calculer le minorant de $G$ pour ce modèle

In [55]:
# TO DO
HBigramme=4*entropie(bigramme['frequency'].tolist())
minGBigramme=(2**HBigramme/4)+1
print(int(minGBigramme))

296254956


Q: combien de temps cela prendra-t-il si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [56]:

print(f"Il faut en moyenne : {((tempsrefBigramme*minGBigramme)/100000)/60} minutes ")

Il faut en moyenne : 2.1218553251705137 minutes 


Q: si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [60]:
# TO DO
H_unif = 8* np.log2(26)
print(f"Entropie maximale car cas uniforme : {H_unif}")

Entropie maximale car cas uniforme : 37.603517745128734


Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [78]:
# TO DO
t_100000=0
letters = monogramme.letters.tolist()
probaUnif=[1/26 for i in range(len(letters))]
nbEssais=100_000
for __ in range(10):
    t = time.time()
    essai= np.random.choice(letters,p=proba,size=(nbEssais,12))
    t_100000 += time.time() - t
print(t_100000/10, 's')
tempsrefUnif = t_100000/10

0.07315473556518555 s


Q: calculer le minorant de $G$ pour ce modèle

In [80]:
# TO DO
HUnif=8*entropie(probaUnif)
minGUnif=(2**HUnif/4)+1
print(int(minGUnif))

52206766144


Q: dans ce cas précis, quelle est la valeur exacte de $G$?

Bal bla espérance 
 (26**8+1)/2

Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

In [81]:
# TO DO
GexactUnif = ((26**8)+1)/2
print(f"Il faut en moyenne : {((tempsrefUnif*GexactUnif)/100000)/60} minutes ")

Il faut en moyenne : 1273.0573906653635 minutes 


Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur:** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [82]:
## Fonction générant un mot de passe
def get_passwd():
    text_hugo = open("texteFrancais.txt","r")
    str_hugo = str(text_hugo.read())

    # On remplace des lettres avec accent avec des lettres sans accent
    str_hugo = str_hugo.replace("Â", "A")
    str_hugo = str_hugo.replace("Ù", "U")
    str_hugo = str_hugo.replace("Ô", "O")
    size_txt = len(str_hugo)

    idx_rand = np.random.randint(size_txt-4)
    #print(idx_rand)

    psswd = str_hugo[idx_rand:idx_rand+4]
    return(psswd)

In [108]:
# Génération du dictionnaire
letters = bigramme.letters.tolist()
proba=bigramme.frequency.tolist()
nbEssais=1_000_000
essai= np.random.choice(letters,p=proba,size=(nbEssais,2))
essai = [essai[i][0]+essai[i][1] for i in range(len(essai))]
df_dictionary = pd.DataFrame(dict(mdp=essai)).mdp.value_counts(normalize=True).to_frame(name="frequency")
df_dictionary.reset_index(inplace=True)
df_dictionary = df_dictionary.rename(columns={'index':'mdp'})

In [111]:
get_passwd()

'DEVR'

In [141]:
df_dictionary[df_dictionary['mdp'] == get_passwd()].index.tolist()[0]

3385

In [143]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
vec_nb_trials = np.full(nb_trial, -1)
for i in range(nb_trial):
    passwrd = get_passwd()
    try:
        index = df_dictionary[df_dictionary['mdp'] ==passwrd].index.tolist()[0]
        vec_nb_trials[i] = index
    except:
        print(passwrd)
    

# To do

UMYR
”TEF
YRIE
DIOC
TLID
PROF
TOÃ™
YRIE
LEOÃ
XGAL
FECT
DITB
IOUF
”TEF
EOÃ™
SQUO
EETF
UINZ
BORD
SBRU
MMYR
™ILY
SMMY
PSPO
EÃ‚M
TDIX
NUMY
MMYR
EYMO
YRIE
YRIE
FECT
ROPQ
BAPT
MMYR
NMMY
Ã‚ME
AIXN
RMMY
MMEY
PROF
‚LEM
OEUV
IILS
AUXQ
NBEA
”TEF


In [ ]:
# Comparaison avec la valeur de G

# To do

## Conclusions 

- Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 
- Définior des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?

**To do**

## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/
